In [16]:
import requests
import numpy as np
import datetime
import pandas as pd
from api_key import api_key
from meetup_api_client import MeetupApiClient
from pymongo import MongoClient
from urllib.parse import urlunparse, urlparse, urlsplit

In [17]:
mc = MongoClient()
db = mc['meetups']
events = db['past_3years_meetup_events']
groups = db['groups']

In [ ]:
start_date = '2015-08-08T00:00:00.000'
end_date = '2018-08-08T00:00:00.000'
mac = MeetupApiClient()
mac.find_past_events(start_date, end_date, groups, events)

In [6]:
header = mac.prev_res

In [ ]:
start_date = '2015-08-08T00:00:00.000'
end_date = '2018-08-08T00:00:00.000'
mac = MeetupApiClient()
mac.find_past_events(start_date, end_date, groups, events)

> /home/cb/Documents/galvanize/assignments/meetup_rsvp_predictor/meetup_api_client.py(57)get_items()
-> while 'Link' in header and header['Link'].partition(',')[0].partition(';')[2] == ' rel="next"':
(Pdb) n
--Return--
> /home/cb/Documents/galvanize/assignments/meetup_rsvp_predictor/meetup_api_client.py(57)get_items()->None
-> while 'Link' in header and header['Link'].partition(',')[0].partition(';')[2] == ' rel="next"':
(Pdb) n
--Return--
> /home/cb/Documents/galvanize/assignments/meetup_rsvp_predictor/meetup_api_client.py(95)get_events()->None
-> self.get_items(urlname + '/events', params, table)
(Pdb) n
> /home/cb/Documents/galvanize/assignments/meetup_rsvp_predictor/meetup_api_client.py(105)find_past_events()
-> for group in groups['data']:
(Pdb) header
*** NameError: name 'header' is not defined


In [3]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
mac.current_res.status_code

In [ ]:
mac.current_res.headers

In [ ]:
mac.current_res.url

In [ ]:
mac.prev_res

In [9]:
header['Link'].partition(',')[0].partition(';')[2] == ' rel="next"'

True

In [11]:
header['Link'].partition(',')[0].partition(';')[0]

'<https://api.meetup.com/Diamond-Way-Buddhism-Seattle/events?scroll=since%3A2017-12-10T16%3A00%3A00.000-08%3A00&key=287212e3b5315867c1a741559133&no_later_than=2018-08-08T00%3A00%3A00.000&status=past&fields=description_images%2C%0A+++++++++++++++++++++++++++++++++++++++event_hosts%2C%0A+++++++++++++++++++++++++++++++++++++++featured%2C%0A+++++++++++++++++++++++++++++++++++++++featured_photo%2C%0A+++++++++++++++++++++++++++++++++++++++fee_options%2C%0A+++++++++++++++++++++++++++++++++++++++group_category%2C%0A+++++++++++++++++++++++++++++++++++++++group_join_info%2C%0A+++++++++++++++++++++++++++++++++++++++group_key_photo%2C%0A+++++++++++++++++++++++++++++++++++++++group_membership_dues%2C%0A+++++++++++++++++++++++++++++++++++++++meta_category%2C%0A+++++++++++++++++++++++++++++++++++++++best_topics%2C%0A+++++++++++++++++++++++++++++++++++++++group_past_event_count%2C%0A+++++++++++++++++++++++++++++++++++++++group_photo%2C%0A+++++++++++++++++++++++++++++++++++++++group_pro_network%2C%0A++

In [ ]:
start_date = datetime.datetime(2015,8,8).strftime('%s') + '000'
end_date = datetime.datetime(2018,8,8).strftime('%s') + '000'
params = {'country': 'us',
          'city': 'seattle',
          'state': 'wa',
          'radius': '100',
          'page': '200',
          'key': api_key()}
mac = MeetupApiClient()
#test = mac.find_past_events(params, start_date, end_date, events)
res = requests.get(mac.meetup_url + 'find/groups', params)

In [ ]:
res.url

Looks like header contains the number of results and a link to get the next page of results

put group data in table

In [ ]:
mac = MeetupApiClient()
params = {'country': 'us',
          'city': 'seattle',
          'state': 'wa',
          'radius': '100'}
res = mac.get_groups(params, groups)

conversion to usfull dataframe

In [ ]:
df = pd.DataFrame()
for event in events.find():
    df = pd.concat([df,  pd.DataFrame(event['data'])], sort=True)

data has some duplicates but some values appear to differ between these duplicates(duplicate is defined as multiple events with the same id)

In [ ]:
df.isnull().sum()

In [6]:
df = pd.DataFrame()
for group in groups.find():
    df = pd.concat([df,  pd.DataFrame(group['data'])], sort=True)

In [7]:
df.duplicated('id').sum()

0

In [ ]:
crummydf = pd.DataFrame(list(groups.find()))
pd.DataFrame(crummydf['data'][0])

In [ ]:
len(df)

status should all = past so it's not a usful feature
maybe_rsvp_count won't be determined until the event has been planned
for a little bit and the creator has no control over it, this feature seems not usful same goes for rating
I'm on the fence about update, it's not something that happens at the time of the event creation but it is something the creator has control of, maybe I'll include it later but for now I'm droping it

In [ ]:
df = df.drop(['maybe_rsvp_count', 
              'status', 
              'rating', 
              'updated', 
              'event_url',
              'headcount'], axis=1)

In [ ]:
df.head()